# Pandas

## Carregamento e Armazenamento de Dados

### Arquivo CSV

In [1]:
import pandas as pd
poke = pd.read_csv('bases/Pokemon.csv')
poke.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


### Arquivo Json

In [ ]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
{"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""
print(type(obj))
print(obj)

In [ ]:
import json
result = json.loads(obj)
result

In [ ]:
asjson = json.dumps(result)
print(type(asjson))
asjson

In [ ]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

### SQL

In [ ]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);"""

In [ ]:
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
('Tallahassee', 'Florida', 2.6, 3),
('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
print(cursor.description)
pd.DataFrame(rows, columns=list(zip(*cursor.description))[0])

In [ ]:
import pandas.io.sql as sql
sql.read_sql_query('select * from test', con)

## Data Preparation

### Combinação de Dados

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd','b'],
                 'data2': range(4)})

pd.merge(df1,df2,on='key') #default inner

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

In [ ]:
pd.merge(df3, df4, how='outer',left_on='lkey',right_on='rkey')

In [2]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [ ]:
import numpy as np
arr = np.arange(12).reshape((3, 4))
print(arr)
np.concatenate([arr, arr], axis=1)

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), 
                index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), 
                index=['a', 'c'],
                columns=['three', 'four'])
print(df1)
print(df2)
pd.concat([df1, df2], axis=1)

### Transformação de Dados

#### Remoção de duplicatas

In [ ]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
print(data)
print(data.duplicated())
data.drop_duplicates()

In [ ]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data.drop_duplicates(['k1', 'k2'], keep='last')

In [ ]:
data2 = pd.DataFrame({'food': ['bacon', 'costela suina', 'bacon', 'picanha',
                           'maminha', 'Bacon', 'picanha', 'presunto','sashimi'],
                  'porcao': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

print(data2)

In [ ]:
meat_to_animal = {'bacon':'porco','costela suina':'porco','picanha':'boi',
                 'maminha':'boi','presunto':'porco','sashimi':"salmao"}

In [ ]:
data2['animal'] = data2['food'].map(str.lower).map(meat_to_animal)
data2

### Replace

In [ ]:
data3 = pd.Series([1., -999., 2., -999., -1000., 3.])
print(data3)
data3.replace(-999,np.nan)
#data3.replace([-999,-1000],np.nan)

### Discretização e Binning

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
group_names = ['Jovem','Meia Idade','Adulto','Idoso']
cats = pd.cut(ages, bins, labels=group_names)
print(cats)
print(pd.value_counts(cats))

In [ ]:
poke.head()

## TODO Section

### Manipulação de Dados usando Pandas

Usando o dataset Pokemon.csv, faça:

    1) Verifique em qual(is) coluna(s) existem valores faltantes
    2) Preencha os valores faltantes da coluna Type 2 com os valores correspondentes da coluna Type 1
    3) Crie um DataFrame a partir dos dados originais contendo apenas pokemons lendários. Imprima os 5 primeiros
    4) Use apply/applymap para passar todos os valores das colunas Name, Type 1 e Type 2 para minúscula
    5) Agrupe os pokemons por Type 1 e retorne uma Série ordenada pela quantidade em ordem decrescente

In [ ]:
# resposta 1


In [ ]:
# resposta 2


In [ ]:
# resposta 3


In [ ]:
# resposta 4


In [ ]:
# resposta 5


In [3]:
import pandas as pd
ages = [1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,
       4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,6,6,6,
       7,7,7,7,7,7,7,7,7,8,8,9,9,9,9]
bins = [1, 4, 7, 10]
group_names = ['1 a 3 anos','4 a 6 anos','7 a 9 anos']
cats = pd.cut(ages, bins, labels=group_names)
print(cats)
print(pd.value_counts(cats))

[NaN, NaN, NaN, NaN, NaN, ..., 7 a 9 anos, 7 a 9 anos, 7 a 9 anos, 7 a 9 anos, 7 a 9 anos]
Length: 69
Categories (3, object): [1 a 3 anos < 4 a 6 anos < 7 a 9 anos]
1 a 3 anos    30
4 a 6 anos    24
7 a 9 anos     6
dtype: int64


In [5]:
import altair as alt
df = pd.DataFrame({'Idade':ages})

In [10]:
alt.Chart(df).mark_bar().encode(
    alt.X("Idade", bin=True),
    alt.Y('count()', title="Quantidade"),
    color = alt.value('grey')
)

alt.Chart(...)